In [2]:
import torch
import torch.nn.functional as F

words = open('names.txt', 'r').read().splitlines()

In [11]:
bichars = set()
chars = set()
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        pre = ch1 + ch2
        pos = ch3
        #bichars.add(pre)
        chars.add(pos)
#bichars = sorted(list(bichars))
chars = sorted(list(chars))

for ch1 in chars:
    for ch2 in chars:
        if ch2 != '.':
            bichars.add(ch1+ch2)

bichars = sorted(list(bichars))

In [17]:
bicharstoi = {s:i for i,s in enumerate(bichars)}
itobichars = {i:s for s,i in bicharstoi.items()}
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}


In [18]:
#Create dataset

xs, ys =[], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = bicharstoi[ch1+ch2]
        ix2 = stoi[ch3]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.nelement() #number of elements in xs
print(f'n of examples: {num}') 

n of examples: 196113


In [19]:
W = torch.rand((len(bichars),len(chars)), requires_grad=True)

In [24]:
xenc = F.one_hot(xs, num_classes = len(bichars)).float()

for k in range(200):
    #forward pass
    logits = xenc @ W

    #Softmax
    counts = (xenc @ W).exp()
    probs = counts / counts.sum(1, keepdims = True)

    loss = -probs[torch.arange(num), ys].log().mean()

    print(loss.item())

    #backward pass
    W.grad = None
    loss.backward()

    W.data += -100 * W.grad
    loss += 0.01 * (W**2).mean()

2.154198169708252
2.1538383960723877
2.1534817218780518
2.153128147125244
2.1527774333953857
2.1524295806884766
2.152085065841675
2.1517434120178223
2.1514041423797607
2.1510682106018066
2.1507349014282227
2.150404214859009
2.150076150894165
2.1497511863708496
2.149428606033325
2.149108648300171
2.1487910747528076
2.1484763622283936
2.1481637954711914
2.1478540897369385
2.1475465297698975
2.1472415924072266
2.1469390392303467
2.146638870239258
2.146340847015381
2.146045207977295
2.145751953125
2.145460844039917
2.145172119140625
2.144885540008545
2.1446011066436768
2.1443188190460205
2.144038677215576
2.1437604427337646
2.143484592437744
2.1432106494903564
2.1429386138916016
2.1426689624786377
2.1424009799957275
2.142134666442871
2.1418707370758057
2.141608476638794
2.1413486003875732
2.141090154647827
2.140833616256714
2.1405789852142334
2.1403262615203857
2.1400749683380127
2.1398255825042725
2.139578342437744
2.1393325328826904
2.1390881538391113
2.138845920562744
2.1386051177978516

In [21]:
def name_generator():
    name = ""

    idbi = torch.randint(0,26,(1,)).item() #first letter randomly generated
    bich = itobichars[idbi]
    name += bich[-1]
    
    while True:
        xenc = F.one_hot(torch.tensor([idbi]), num_classes=len(bichars)).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)

        idch = torch.multinomial(p, num_samples = 1, replacement = True).item() #samples from probability distribution p (returns index)
        ch = itos[idch]
        
        if len(name) < 3:
            while ch == '.':
                idch = torch.multinomial(p, num_samples = 1, replacement = True).item() #samples from probability distribution p (returns index)
                ch = itos[idch]
                bich = bich[-1] + ch
                idbi = bicharstoi[bich]

        if ch == ".":
            return name
        
        name += ch

        bich = bich[-1] + ch
        idbi = bicharstoi[bich]


In [23]:
[name_generator() for _ in range(10)]

['orae',
 'ugtgaxigh',
 'tavitaylynn',
 'prey',
 'harickenai',
 'zaraelyleala',
 'ajyokin',
 'hani',
 'que',
 'olugjhayn']